In [1]:
import sqlite3
import re
import calendar # for isleap
import datetime
import pandas as pd

In [23]:
from functools import wraps
import inspect

def get_class_that_defined_method(meth):
    if inspect.ismethod(meth):
        for cls in inspect.getmro(meth.__self__.__class__):
            if cls.__dict__.get(meth.__name__) is meth:
                return cls
        meth = meth.__func__ # fallback to __qualname__ parsing
    if inspect.isfunction(meth):
        cls = getattr(inspect.getmodule(meth),
                      meth.__qualname__.split('.<locals>', 1)[0].rsplit('.', 1)[0])
        if isinstance(cls, type):
            return cls
        
def unimplemented(func):
    @wraps(func)
    def tmp(*args, **kwargs):
        #func_name = sys._getframe().f_code.co_name
        #func_name = inspect.currentframe().f_code.co_name
        #func_name = inspect.stack()[0][0].f_code.co_name
        #func_name = inspect.stack()[0][3]
        caller = inspect.stack()[1][3]
        cls = get_class_that_defined_method(func)
        print("Warning: Unimplemented function %s() called by %s:%s()" % (func.__name__, cls, caller))
        return func(*args, **kwargs)
    return tmp

class Book(object):
    def __init__(self, gnc_db):
        self.db = sqlite3.connect(gnc_db)
        self._getCommodities()
        self._createCommodityObjects()
        self._getBookRootAccount()
        self._getBookAccounts()
        self._createAccountObjects()
        self._getCommodityPrices()
        self._getTransactions()
        self._getSplits()
    
    def _getCommodities(self):
        self.commDF = pd.read_sql_query("SELECT * FROM commodities;", self.db)
        
    def _createCommodityObjects(self):
        self.commodities = []
        for i in range(self.commDF.shape[0]):
            comm = self.commDF.iloc[[i]]
            c = Commodity(self, comm)
            self.commodities.append(c)
            
    def _getCommodityPrices(self):
        self.pricesDF = pd.read_sql_query("SELECT * FROM prices;", self.db)
                
    def _getBookRootAccount(self):
        """Obtain the root account guid of the gnucash book.
        
        Assumptions:
        - There is only one root account in the book
        """
        r = self.db.execute("SELECT root_account_guid FROM books;").fetchone()
        if r is None:
            raise ValueError("No valid root_account found in the gnucash book")
        else:
            self.rootAccountGuid = r[0]

    def _getBookAccounts(self):
        self.accDF = pd.read_sql_query("SELECT * FROM accounts;", self.db)
        
    def _createAccountObjects(self):
        self.root = Account(self, self.accDF.loc[self.accDF.guid == self.rootAccountGuid], parent=None)
        self.accounts = [self.root]
        for child, parentGuid in self._iterAccountTree(self.root.guid):
            acc = Account(self, child, self.getAccountByGuid(parentGuid))
            self.accounts.append(acc)

    def _iterAccountTree(self, parentGuid):
        children = self._findChildrenOf(parentGuid)
        for i in range(children.shape[0]):
            child = children.iloc[[i]]
            yield (child, parentGuid)
            yield from self._iterAccountTree(child.iloc[0].guid)
        
    def _findChildrenOf(self, parentGuid):
        ch = self.accDF.loc[self.accDF.parent_guid == parentGuid]
        return ch
    
    def _getTransactions(self):
        self.txDF = pd.read_sql_query("SELECT * FROM transactions;", self.db)
    
    def _getSplits(self):
        self.splitsDF = pd.read_sql_query("SELECT * FROM splits;", self.db)
    
    def getAccountByGuid(self, accGuid):
        for a in self.accounts:
            if a.guid == accGuid:
                return a
        return None
    
    def getAccountByName(self, accName, strict=True):
        for a in self.accounts:
            if a.name == accName:
                return a
        return None
    
    def getCommodityByGuid(self, commGuid):
        for c in self.commodities:
            if c.guid == commGuid:
                return c
        return None
    
    def getCommodityByName(self, commName):
        for c in self.commodities:
            if c.name == commName:
                return c
        return None
    
    @unimplemented
    def getCommodityPrice(self, comm, curr, refDate):
        print(comm)
        print(curr)
        if not self._isGuid(comm):
            comm = self.getCommodityByName(comm).guid
        if not self._isGuid(curr):
            curr = self.getCommodityByName(curr).guid
        prices = self.pricesDF.query("commodity_guid == '%s' and currency_guid == '%s'" % (comm, curr))
        print(prices)
        prices = prices.sort_values(by="date", axis=1)
        print(prices)
        pass
            
    def _getTxByQuery(self, queryString):
        res = self.txDF.query(queryString)
        return res
    
    def _getSplitsByTx(self, txGuid):
        res = self.splitsDF[self.splitsDF.tx_guid == txGuid]
        return res
    
    def _isGuid(self, guid):
        if not isinstance(guid, str):
            return False
        if not re.match(r"[0-9a-fA-F]{32}", guid):
            return False
        return True
    
    def _printAccounts(self, parent, level=0):
        print(level*" " + parent.name)
        for c in parent.children:
            self._printAccounts(c, level+3)
            
    @unimplemented
    def reportCf(self, accounts, per_beg, per_end, currency="CHF"):
        if isinstance(per_beg, str):
            per_beg = GncDate(per_beg)
        if per_end == "" or per_end is None:
            per_end = "2100-01-01"
        if isinstance(per_end, str):
            per_end = GncDate(per_end)
        if per_end.datetime() < per_beg.datetime():
            raise ValueError("Period start is > period end")
            
        # Convert the account names to guid's
        accGuids = list(map(lambda x: self.getAccountByName(x).guid, accounts))
        
        # Get the guids of all trading accounts
        tradingGuids = list(map(lambda x: x.guid, filter(lambda x: x.type == Account.TRADING, book.accounts)))
        
        # Get report's currency guid
        currGuid = self.getCommodityByName(currency).guid
        
        # Get all tx in the given period
        tx_for_period = self._getTxByQuery('post_date >= "%s" and post_date <= "%s"' % 
                                           (per_beg.to_gnc(), per_end.to_gnc()))
        if tx_for_period.empty:
            return None
        
        # Empty dataframe that will accumulate the final account-trimmed splits
        splits = pd.DataFrame()
    
        #print(tx_for_period)
        for _, tx in tx_for_period.iterrows():
            #print(tx)
            tx_splits = self._getSplitsByTx(tx.guid)
        
            # Delete splits belonging to the accounts in question
            nb_tx_splits = tx_splits.size;
            tx_splits = tx_splits[~tx_splits.account_guid.isin(accGuids)]
            
            # If no splits have been deleted, the tx is to be dumped
            # as it does not concern the required accounts
            if tx_splits.size == nb_tx_splits:
                continue

            # Delete all splits belonging to TRADING accounts
            tx_splits = tx_splits[~tx_splits.account_guid.isin(tradingGuids)]

            if tx_splits.empty:
                continue
            
            # Apply exchange rate if needed
            if tx.currency_guid != currGuid:
                xrate_num, xrate_denom = self.getCommodityPrice(tx.currency_guid, 
                                                                currGuid, 
                                                                per_end)
                print(xrate_num, xrate_denom)
                tx_splits.value_num *= xrate_num
                tx_splits.value_denom *= xrate_denom
                tx.currency_guid = currGuid

            splits = splits.append(tx_splits)
    
        return splits
        
class Account(object):
    ROOT = 1
    ASSET = 2
    CASH = 3
    BANK = 4
    LIABILITY = 5
    CREDIT = 6
    TRADING = 7
    
    def __init__(self, book, accDF, parent):
        acc = accDF.iloc[0]
        self.book = book
        self.guid = acc.guid
        self.name = acc['name']
        self.type = self._type(acc.account_type)
        self.parent_guid = acc.parent_guid
        self.parent = parent
        self.description = acc.description
        self.commodity = self.book.getCommodityByGuid(acc.commodity_guid)
        self.hidden = acc.hidden
        self.placeholder = (acc.placeholder == 1)
        self.children = [] # to be completed via addChild
        if parent is not None:
            parent.addChild(self)
        
    def _type(self, accType):
        if accType == 'ROOT': return self.ROOT
        if accType == 'ASSET': return self.ASSET
        if accType == 'CASH': return self.ROOT
        if accType == 'BANK': return self.BANK
        if accType == 'LIABILITY': return self.LIABILITY
        if accType == 'CREDIT': return self.CREDIT
        if accType == 'TRADING': return self.TRADING
        
    def addChild(self, child):
        if not isinstance(child, Account):
            ValueError("Non Account-type passed for child")
        self.children.append(child)
        
class Commodity(object):
    CURRENCY = 1
    def __init__(self, book, commDF):
        comm = commDF.iloc[0]
        self.book = book
        self.guid = comm.guid
        self.name = comm.mnemonic
        self.fullname = comm.fullname
        self.type = self._type(comm.namespace)
        self.iscurrency = comm.namespace == "CURRENCY"
        
    def _type(self, commType):
        if commType == "CURRENCY": return self.CURRENCY
        
class GncDate(object):
    class GMT1(datetime.tzinfo):
        def utcoffset(self, dt):
            return datetime.timedelta(hours=1) + self.dst(dt)
        
        def dst(self, dt):
            # DST starts last Sunday in March
            d = datetime.datetime(dt.year, 4, 1)   # ends last Sunday in October
            self.dston = d - datetime.timedelta(days=d.weekday() + 1)
            d = datetime.datetime(dt.year, 11, 1)
            self.dstoff = d - datetime.timedelta(days=d.weekday() + 1)
            if self.dston <=  dt.replace(tzinfo=None) < self.dstoff:
                return datetime.timedelta(hours=1)
            else:
                return datetime.timedelta(0)
            
        def tzname(self,dt):
            return "GMT +1"

    def __init__(self, isodate):
        self.date = None

        try:
            self.date = datetime.datetime.strptime(isodate, "%Y-%m-%d")
        except ValueError:
            try:
                self.date = datetime.datetime.strptime(isodate, "%y-%m-%d")
            except ValueError:
                try:
                    self.date = datetime.datetime.strptime(isodate, "%y-%m-%d%z")
                except ValueError:
                    self.date = datetime.datetime.strptime(isodate, "%Y-%m-%d%z")
    
        if self.date.tzinfo == None:
            gmt1 = self.GMT1()
            self.date = self.date.replace(tzinfo=gmt1)
    
    def to_gnc(self):
        d = self.date
        d = d - d.utcoffset()
        ret = "{:04}{:02}{:02}{:02}{:02}{:02}".format(d.year, d.month, d.day, d.hour, d.minute, d.second)
        return ret
    
    def datetime(self):
        return self.date
    
    @unimplemented
    def from_gnc(gncdate):
        m = re.match(r"(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})(?P<hr>\d{2})(?P<min>\d{2})(?P<sec>\d{2})", gncdate)
        if not m:
            raise ValueError("The input is not in gnucash-date format")
        if m.group('min') != "00" or m.group('sec') != "00":
            raise ValueError("Still don't know how to handle gnucash-dates with non-zero min/sec parts")
        return GncDate("%s-%s-%s" % (m.group('year'), m.group('month'), m.group('day')))
        pass

In [24]:
d = GncDate("2017-06-01")
d.to_gnc()

'20170531220000'

In [25]:
d =GncDate.from_gnc("20170531220000")
d.to_gnc()

'20170530220000'

In [4]:
book = Book("money.gnucash.sql.gnucash")

In [5]:
accs_cf = ['Private', 'Private-Bills']
book.reportCf(accs_cf, GncDate("2017-02-28"), GncDate("2017-03-27"))

,guid,tx_guid,account_guid,memo,action,reconcile_state,reconcile_date,value_num,value_denom,quantity_num,quantity_denom,lot_guid
13411,e79350885060b1f2e636dd6f9e1973b2,e09ec9e5c6ab2b72d106aecd9a39017c,a965e2c6bde903c170ac1f1f47ea6440,food,,n,None,2530,100,2530,100,None
13412,1265066c4054af5543a762c75346b371,e09ec9e5c6ab2b72d106aecd9a39017c,c0cd223dbf0a570d9cfd90b9749cde8a,wine,,n,None,1390,100,1390,100,None
13414,97cd4b9ecd8b47ddb9d644a211ec416b,0544fed87dadad4e672fc365ca057b4a,3977c26d2e4ae9492b36ff26f9af55fe,,,n,None,300,100,300,100,None
13417,7c283c6c7648289727460dc23dbb882d,8df9aacbe22e6d9a356aaa1fd1239e34,b58b3d0c66017aebf3648c091265f7d7,,,n,None,-813410,100,-813410,100,None
13419,9bf3196c6b3b93a46fbcb88ddf18e995,be3fc7483c99253089ab5787396ebbb6,b58b3d0c66017aebf3648c091265f7d7,,,n,None,-12375,100,-12375,100,None
13420,493d6aaea510b77f2b25791f0f942580,642cac02c87fd8fd1d894e5caa393f41,065cf464ceffdd68bcbb36b983fee364,,,n,None,173000,100,173000,100,None
13422,e4753844e3c48a73e120189fc40b8725,5991d75ad8509ae2959cb66c1a65d770,f84ad2440286f60aaa6a0fad48a131dd,,,n,None,46235,100,46235,100,None
13424,04a97f857cc5f7f5d9a50267b360dce9,de5f353f4ec76626bde5edd7936a16f6,f84ad2440286f60aaa6a0fad48a131dd,,,n,None,43950,100,43950,100,None
13426,8729037e75fb1409433c6819ea889bd4,915747608c4ca5bea3cb3150a13ce002,12885edd8630cc4819c5c7e38fa7cff4,,,n,None,11275,100,11275,100,None
13428,ef4aeb55fab36f89dd3ef62bf5f139e6,aecc98a64e3a685aeb0f1c30dc6b027f,1706edb22c7673f37843951230899f81,,,n,None,2500,100,2500,100,None


In [12]:
import sys
sys.version

'3.4.3 (default, May  5 2015, 17:58:45) \n[GCC 4.9.2]'